In [ ]:
from IPython.display import HTML

HTML(r"""
<style>
body {
  font-family: 'Segoe UI', sans-serif;
  background: #121212 !important;
  color: #EEE !important;
}
.grid-dashboard {
  display: grid;
  grid-template-columns: 1.3fr 2fr;
  gap: 1rem;
  margin: 1rem 0;
}
.control-card .widget-box,
.output-card .widget-box {
  background: #1E1E1E !important;
  border-radius: 1rem !important;
  padding: 1.2rem !important;
  box-shadow: 0 6px 20px rgba(0,0,0,0.6) !important;
  transition: background 0.3s ease, transform 0.2s ease;
}
.control-card .widget-box:hover,
.output-card .widget-box:hover {
  background: #2A2A2A !important;
  transform: translateY(-2px);
}
.jp-TabBar-tab {
  background: #1E1E1E !important;
  color: #AAA !important;
  border-radius: 0.5rem 0.5rem 0 0 !important;
  margin-right: 0.3rem !important;
  padding: 0.6em 1em !important;
}
.jp-TabBar-tab.jp-mod-active {
  background: #333333 !important;
  color: #FFF !important;
  box-shadow: inset 0 -3px 0 #1DB954 !important;
}
.insight-box {
  background: #333333 !important;
  border-left: 4px solid #1DB954;
  padding: 1rem !important;
  margin-top: 1rem !important;
  font-style: italic;
  color: #DDD !important;
  border-radius: 0.5rem !important;
  animation: fadeIn 0.5s ease-out;
}
@keyframes fadeIn {
  from { opacity: 0; transform: translateY(10px); }
  to   { opacity: 1; transform: translateY(0); }
}
</style>
""")


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML

def calc_coupon_price(F, C, r, T):
    return sum(C/(1+r)**t for t in range(1, T+1)) + F/(1+r)**T

def calc_zero_price(F, r, T):
    return F/(1+r)**T

def calc_macaulay_duration(F, C, r, T):
    P = calc_coupon_price(F, C, r, T)
    numerator = sum(t * C/(1+r)**t for t in range(1, T+1)) + T * F/(1+r)**T
    return numerator / P

def calc_modified_duration(D, r):
    return D / (1 + r)

def calc_convexity(F, C, r, T):
    P = calc_coupon_price(F, C, r, T)
    cashflows = [C]*(T-1) + [C+F]
    return sum(t*(t+1)*cf/(1+r)**(t+2) for t, cf in zip(range(1, T+1), cashflows)) / P


# 💵 Bond Pricing & Duration: Interactive Dashboard

**Core Equations**

1. **Coupon Bond Price**  
   $$P = \sum_{t=1}^{T} \frac{C}{(1+r)^t} + \frac{F}{(1+r)^T}$$

2. **Zero‐Coupon Price**  
   $$P = \frac{F}{(1+r)^T}$$

3. **Macaulay Duration**  
   $$D = \frac{\sum_{t=1}^{T} t\,\frac{CF_t}{(1+r)^t}}{P}$$

4. **Modified Duration**  
   $$D_{\mathrm{mod}} = \frac{D}{1+r}$$

5. **Convexity**  
   $$\mathrm{Convexity} = \frac{\sum_{t=1}^{T} t(t+1)\,\frac{CF_t}{(1+r)^{t+2}}}{P}$$


In [ ]:
def bond_dashboard_pro():
    style    = {'description_width':'160px'}
    ctrl_cls = {'_dom_classes':['control-card']}
    out_cls  = {'_dom_classes':['output-card']}

    # ── Widgets ─────────────────────────────────────
    bond_type = widgets.Dropdown(
        options=['Coupon Bond','Zero-Coupon Bond'], value='Coupon Bond',
        description='Bond Type:', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    face_value = widgets.IntSlider(
        value=1000, min=500, max=5000, step=100,
        description='Face Value ($):', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    coupon_rate = widgets.FloatSlider(
        value=0.05, min=0.0, max=0.15, step=0.0025,
        description='Coupon Rate (%):', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    market_rate = widgets.FloatSlider(
        value=0.05, min=0.01, max=0.15, step=0.0025,
        description='Market YTM (%):', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    spot_rate = widgets.FloatSlider(
        value=0.05, min=0.01, max=0.15, step=0.0025,
        description='Spot Rate (%):', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    use_spot = widgets.Checkbox(
        value=False, description='Use Spot Rate',
        style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    maturity = widgets.IntSlider(
        value=10, min=1, max=40, step=1,
        description='Maturity (Yrs):', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    compare_tenors = widgets.SelectMultiple(
        options=list(range(1,41)), value=[10,20,30],
        description='Compare Tenors:', style=style, layout=widgets.Layout(width='100%'),
        **ctrl_cls
    )
    play = widgets.Play(
        value=market_rate.value, min=1, max=15, step=1, interval=200,
        description='Animate Rate'
    )
    widgets.jslink((play, 'value'), (market_rate, 'value'))
    scenario = widgets.ToggleButtons(
        options=['Flat=5%','Invert=10%','Shock+2%'],
        description='Scenario:', style=style,
        **ctrl_cls
    )

    # ── Output Panes ─────────────────────────────────
    out_sum, out_cf, out_curve, out_multi, out_ctx, out_ins = [
        widgets.Output(**out_cls) for _ in range(6)
    ]

    # ── Scenario Logic ───────────────────────────────
    def on_scenario(change):
        v = change['new']
        if v == 'Flat=5%':
            market_rate.value = 0.05
        elif v == 'Invert=10%':
            market_rate.value = 0.10
        else:
            market_rate.value = min(0.15, market_rate.value + 0.02)
    scenario.observe(on_scenario, names='value')

    # ── Update Callback ───────────────────────────────
    def update(bt, F, cr, mr, sr, us, T, tens):
        r     = sr if us else mr
        C     = F*cr if bt=='Coupon Bond' else 0
        P     = calc_coupon_price(F, C, r, T) if C else calc_zero_price(F, r, T)
        D     = calc_macaulay_duration(F, C, r, T) if C else T
        D_mod = calc_modified_duration(D, r)
        Cv    = calc_convexity(F, C, r, T) if C else 0
        mD    = calc_macaulay_duration(F, C, 0.041, 10)
        rates = np.linspace(0.01,0.15,200)
        P10   = calc_zero_price(1000, 0.041, 10)

        # Summary
        with out_sum:
            out_sum.clear_output()
            display(HTML(f"""
<div class="output-card">
<b>📋 Results</b><br>
Type: {bt}<br>
Price: <b>${P:,.2f}</b><br>
Macaulay Dur: <b>{D:.2f} yrs</b><br>
Mod Dur: <b>{D_mod:.2f} yrs</b><br>
Convexity: <b>{Cv:.2f}</b><br>
Market-Implied Dur (10y@4.1%): <b>{mD:.2f} yrs</b><br>
Coupon: <b>${C:.2f}</b>
</div>
"""))

        # PV Cash Flows
        with out_cf:
            out_cf.clear_output()
            yrs = np.arange(1, T+1)
            cf  = [C]*(T-1) + [C+F] if C else [0]*(T-1)+[F]
            pv  = [cf_i/(1+r)**t for t,cf_i in zip(yrs, cf)]
            fig1 = go.Figure()
            fig1.add_trace(go.Scatter(
                x=yrs, y=pv, mode='lines+markers',
                line=dict(color='teal', width=4, shape='spline'),
                marker=dict(size=8, color='lightgreen'),
                fill='tozeroy', fillcolor='rgba(0,255,200,0.2)',
                hovertemplate='Year %{x}<br>PV: $%{y:.2f}<extra></extra>'
            ))
            fig1.update_layout(
                template='plotly_dark',
                title_text='Present Value of Cash Flows',
                font_family='Segoe UI',
                font_size=14,
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                margin=dict(t=40,l=30,r=30,b=30),
                height=300
            )
            display(fig1)

        # Price vs Yield Curve
        with out_curve:
            out_curve.clear_output()
            prices = [calc_coupon_price(F, C, rr, T) if C else calc_zero_price(F, rr, T)
                      for rr in rates]
            fig2 = go.Figure()
            fig2.add_trace(go.Scatter(
                x=rates*100, y=prices, mode='lines',
                line=dict(color='#FF6361', width=4),
                fill='tozeroy', fillcolor='rgba(255,99,97,0.2)',
                hovertemplate='Yield: %{x:.2f}%<br>Price: $%{y:.2f}<extra></extra>'
            ))
            fig2.update_layout(
                template='plotly_dark',
                title_text='Price vs Yield',
                xaxis_title='Yield (%)',
                yaxis_title='Price ($)',
                font_family='Segoe UI',
                font_size=14,
                margin=dict(t=40,l=30,r=30,b=30),
                height=300
            )
            display(fig2)

        # Compare Tenors
        with out_multi:
            out_multi.clear_output()
            fig3 = go.Figure()
            palette = px.colors.qualitative.Dark24
            for i, ten in enumerate(tens):
                pr = [calc_coupon_price(F, C, rr, ten) if C else calc_zero_price(F, rr, ten)
                      for rr in rates]
                fig3.add_trace(go.Scatter(
                    x=rates*100, y=pr, mode='lines', name=f'{ten} yrs',
                    line=dict(color=palette[i % len(palette)], width=3)
                ))
            fig3.update_layout(
                template='plotly_dark',
                title_text='Yield Curves for Multiple Tenors',
                xaxis_title='Yield (%)',
                yaxis_title='Price ($)',
                font_family='Segoe UI',
                font_size=14,
                margin=dict(t=40,l=30,r=30,b=30),
                height=300
            )
            display(fig3)

        # Live Context
        with out_ctx:
            out_ctx.clear_output()
            display(HTML(f"""
<div class="output-card">
🌐 10Y ZCB @4.1% → <b>${P10:,.2f}</b>
</div>
"""))

        # Insight Box
        with out_ins:
            out_ins.clear_output()
            display(HTML(f"""
<div class="insight-box">
ℹ️ At r={r:.2%}, Price=${P:,.2f}, Duration={D:.2f} yrs
</div>
"""))

    # Assemble & display
    controls = widgets.VBox([
        bond_type, face_value, coupon_rate, market_rate, spot_rate,
        use_spot, maturity, compare_tenors, play, scenario
    ], _dom_classes=['control-card'])
    outputs = widgets.Tab([
        out_sum, out_cf, out_curve, out_multi, out_ctx, out_ins
    ], _dom_classes=['output-card'])
    for idx, title in enumerate([
        'Summary','Cash Flows','Price Curve','Compare','Context','Insight'
    ]):
        outputs.set_title(idx, title)
    display(widgets.HBox([controls, outputs], _dom_classes=['grid-dashboard']))
    widgets.interactive_output(update, {
        'bt': bond_type, 'F': face_value, 'cr': coupon_rate,
        'mr': market_rate, 'sr': spot_rate, 'us': use_spot,
        'T': maturity, 'tens': compare_tenors
    })


In [ ]:
bond_dashboard_pro()
